# Extraktion von Informationen aus Texten mit spaCy

## Unser Datensatz und unsere Aufgabe

Unser Datensatz wurde von E. Leitner, G. Rehm und J. Moreno-Schneider  in

>
> **[Fine-grained Named Entity Recognition in Legal Documents.](https://link.springer.com/chapter/10.1007/978-3-030-33220-4_20)**

vorgestellt und ist verfügbar unter
[github](https://github.com/elenanereiss/Legal-Entity-Recognition). Der Datensatz besteht aus **Urteilen** mehrerer deutscher Gerichte mit Annotationen aller Erwähnungen von Gesetzen, Urteilen, Literatur etc.:

<img src="img/legal_ner.png" width="600px" align="center"/>

Der gesamte Datensatz umfasst 66.723  Sätze. Wir wählen daraus

- Urteile des Bundesarbeitsgerichts (BAG) zum **Training** und
- Urteile des Bundesgerichtshofes (BGH) zur **Validatierung**.

Die folgenden Histogramme zeigen die Verteilungen

- der Satzlängen und 
- der Anzahl der Annotationen pro Klasse, wobei 'O' die "leere" Klasse ist:

![](img/data.svg)

Unsere Aufgabe wird es sein, jedes Token in einem Satz mit der richtigen Annotation zu versehen. Diese Aufgabe bezeichnet man als **named entity recognition** (**NER**).


## Die NLP-Bibliothek spaCy

Die Python-Biliothek [spaCy](https://spacy.io) bietet ***"industrial-strength natural language processing"*** und erlaubt uns, die obige Aufgabe zu lösen, ohne irgendetwas von NLP zu verstehen!
Zuerst müssen wir ein [**Sprachmodell**](https://spacy.io/models) laden:

In [ ]:
# !python -m spacy download de_core_news_md

Die obige Aufgabe werden wir mit wenigen Kommandos auf der Kommandozeile lösen. Vorher werfen wir aber einen kleinen Blick auf "klassisches" NLP mit spaCy:

In [ ]:
import spacy

nlp = spacy.load('de_core_news_md')

doc = nlp('Es war des Mondes heller Schein, der fiel durch Fensterscheiben rein.')

Wendet man das Sprachmodell auf einen Text an, so wird dieser in eine Folge von Token zerlegt und für diese Token werden

- die Wortart (Part-of-Speech),
- die Grundform und
- der Wortvektor

bestimmt:

In [ ]:
import pandas as pd

pd.DataFrame({'text': [token.text for token in doc],
              'pos': [token.pos_ for token in doc],
             'lemma': [token.lemma_ for token in doc],
             'vector': [token.vector for token in doc]})

Das Ergebnis ist nur zum Teil korrekt... SpaCy bietet aber noch viel mehr.

## Schritt 1: Daten aufbereiten

Wir haben bereits von [github](https://github.com/elenanereiss/Legal-Entity-Recognition) die Daten für den BGH und das BAG heruntergeladen und im Verzeichnis `data/legal/01_raw` abgelegt. Die Dateien enthalten Beispielsätze mit Annotationen im conll-Format: pro Zeile ein Wort und die zugehörige Annotation:

In [ ]:
!head -n 20 data/legal/01_raw/bgh.conll

SpaCy kann dieses Standardformat in das eigene Format wie folgt umwandeln:

In [ ]:
!python -m spacy convert --converter ner data/legal/01_raw/bag.conll data/legal/02_train
!python -m spacy convert --converter ner data/legal/01_raw/bgh.conll data/legal/03_val

## Schritt 2: Konfiguration

Um mit spaCy per Kommandozeile ein Modell zu trainieren, muss man seit Version 3 eine Konfiguration anlegen:

In [ ]:
!python -m spacy init config data/legal/config_empty.cfg --lang de --pipeline 'ner' --optimize 'efficiency' --force

In der angelegten Konfigurationsdatei müssen wir am Anfang unter `paths.train` und `paths.dev` die Pfade zu den Trainings- und Validierungsdaten eintragen. Dies ist in der Datei `data/legal/config.cfg` bereits geschehen.

## Schritt 3: Daten prüfen

Wir lassen spaCy nun die Daten einmal kurz prüfen:

In [ ]:
!python -m spacy debug data data/legal/config.cfg --verbose

## Schritt 4: Modell trainieren und auswerten

Nun können wir ein NER-Modell wie folgt trainieren:

In [ ]:
!python -m spacy train data/legal/config.cfg -o data/legal/04_models

Auswerten können wir das trainierte Modell auf unseren Test-Daten wie folgt:

In [ ]:
!python -m spacy evaluate data/legal/04_models/model-best data/legal/03_val

Diese Tabelle zeigt pro Annotationstyp die Scores Precision (P), Recall (R) und den F1-Score (F). Die Werte variieren stark und sind noch nicht so gut. Wir haben aber auch

- nur sehr kurz trainiert und
- eine sehr unausgewogene Datenbasis: einige Annotationen treten extrem selten auf, sodass sie auch kaum gelernt werden können.

## Schritt 5: Modell verwenden

Um das trainierte Modell anzuwenden, verwenden wir die [Python-API](https://spacy.io/api) von [spaCy](https://spacy.io):

In [ ]:
import spacy
MODEL_PATH = 'data/legal/04_models/model-best'
nlp = spacy.load(MODEL_PATH)

Das Modell kann nun wie folgt auf Texte angewendet werden:

In [ ]:
import pandas as pd

sample = """Trotz der zweifelhaften Bewertung von MDMA als "harte Droge"
( vgl. BGH , Beschluss vom 3. Februar 1999 - 5 StR 705/98 ,
juris Rn. 2 ; zum Meinungsstand Patzak in Körner / Patzak / Volkmer
, BtMG , 8. Aufl. , Vorbem. zu §§ 29 ff. Rn. 213 mwN ; Weber , BtMG ,
5. Aufl. , § 1 Rn. 364 mwN ) hat der Strafausspruch Bestand ,
da die verhängte Rechtsfolge jedenfalls angemessen ist 
(§ 354 Abs. 1a Satz 1 StPO) ."""

doc = nlp(sample)
print(pd.DataFrame.from_records([{'Annotation': ent.label_, 'Text': ent.text} for ent in doc.ents]))